Import The Dependency

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from windrose import WindroseAxes
import os


Load The Raw Data

In [ ]:
# Reload the raw dataset to start fresh
df = pd.read_csv('../data/raw/togo-dapaong_qc.csv')



initial explaratory data analaysis

In [ ]:
print("First 10 rows of raw data:")
print(df[['GHI', 'DNI', 'DHI']].head(10))

print("\nRandom sample of 10 rows:")
print(df[['GHI', 'DNI', 'DHI']].sample(10))




In [ ]:
# How many rows do we actually have?
print("Data shape:", df.shape)

# Show summary of selected columns
print(df[['GHI', 'DNI', 'DHI', 'Tamb', 'WS']].describe())

In [ ]:
# Check how many rows have negative values for each column
for col in ['GHI', 'DNI', 'DHI']:
    negative_count = len(df[df[col] < 0])
    print(f"Number of rows with {col} < 0: {negative_count} ({negative_count / len(df) * 100:.2f}%)")

In [ ]:
# Summary statistics of all numeric columns
summary_stats = df.describe()
display(summary_stats)

In [ ]:
# Missing values count per column
missing_values = df.isna().sum()
display(missing_values)

In [ ]:
# Columns with more than 5% missing values
missing_percent = (missing_values / len(df)) * 100
cols_over_5pct_null = missing_percent[missing_percent > 5]
print("Columns with >5% missing values:")
print(cols_over_5pct_null)

step-3 Pre-Proccesing

In [ ]:
# Drop columns with more than 5% missing values
df = df.drop(columns=["Comments"])

set nagative values to zero

In [ ]:
irr_cols = ['GHI', 'DNI', 'DHI']
for col in irr_cols:
    df[col] = df[col].clip(lower=0)  # Set negative values to 0
    print(f"After clipping, number of rows with {col} < 0: {len(df[df[col] < 0])}")

In [ ]:
#Convert Timestamp to datetime and set as index
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.set_index('Timestamp')

In [ ]:



# Set up a grid for plots
cols_to_check = ['GHI', 'DNI', 'DHI', 'Tamb', 'WS']
plt.figure(figsize=(15, 10))

for i, col in enumerate(cols_to_check):
    plt.subplot(2, 3, i + 1)
    sns.histplot(df[col], bins=50, kde=True)
    plt.title(f'Distribution of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


Next Step: Outlier Detection & Basic Cleaning (Detailed)
Objective: Detect outliers using Z-scores (|Z| > 3) and handle them 

Plan:

Compute Z-scores for GHI, DNI, DHI, ModA, ModB, WS, and WSgust.
Flag rows with |Z| > 3 as outliers.
Impute outliers with the median value of each column.

In [ ]:


# Define key columns for outlier detection
key_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Detect and handle outliers using Z-scores
for col in key_cols:
    if col in df.columns:
        # Calculate Z-scores, ignoring NaN
        z_scores = stats.zscore(df[col].dropna())
        # Create a mask for outliers (|Z| > 3)
        outlier_mask = abs(z_scores) > 3
        # Get the original indices of outliers
        outlier_indices = df.index[np.where(outlier_mask)[0]]
        print(f"Number of outliers for {col}: {len(outlier_indices)}")
        # Impute outliers with the median (only for non-NaN values)
        df.loc[outlier_indices, col] = df[col].median()

# Verify the shape and summary statistics after outlier handling
print("\nData shape after outlier handling:", df.shape)
print("\nSummary Statistics after outlier handling:")
print(df[['GHI', 'DNI', 'DHI', 'Tamb', 'WS']].describe())

EDA

In [ ]:

# How many rows do we actually have?
print("Data shape:", df.shape)

# Show summary of selected columns
print(df[['GHI', 'DNI', 'DHI', 'Tamb', 'WS']].describe())



# Summary statistics of all numeric columns
summary_stats = df.describe()
display(summary_stats)
# Check how many rows have negative values for each column
for col in ['GHI', 'DNI', 'DHI']:
    negative_count = len(df[df[col] < 0])
    print(f"Number of rows with {col} < 0: {negative_count} ({negative_count / len(df) * 100:.2f}%)")

# Check data types
print("\nChecking Column Data Types:")
print(df.dtypes)

# Missing values count per column
missing_values = df.isna().sum()
display(missing_values)

# Summary of categorical columns (text output)
print("\nSummary of categorical columns\n")
categorical_columns = df.select_dtypes(include=['object']).columns
for col in categorical_columns:
    print(f"Value Counts of {col}:")
    print(df[col].value_counts())

# Columns with more than 5% missing values
missing_percent = (missing_values / len(df)) * 100
cols_over_5pct_null = missing_percent[missing_percent > 5]
print("Columns with >5% missing values:")
print(cols_over_5pct_null)    



Visualization

Next Step: Time Series Analysis
Objective: Perform time series analysis on GHI, DNI, DHI, and Tamb to observe patterns 

Plan:
Plot line charts for GHI, DNI, DHI, and Tamb over time.
 Analyze patterns by month and hour.
 Identify anomalies (e.g., unexpected peaks in GHI).

In [ ]:

# Plot line charts for GHI, DNI, DHI, and Tamb over time
plt.figure(figsize=(12, 8))

# Plot GHI
plt.subplot(2, 2, 1)
plt.plot(df.index, df['GHI'], label='GHI', color='blue')
plt.title('GHI over Time')
plt.xlabel('Timestamp')
plt.ylabel('GHI (W/m²)')
plt.xticks(rotation=45)
plt.grid(True)

# Plot DNI
plt.subplot(2, 2, 2)
plt.plot(df.index, df['DNI'], label='DNI', color='orange')
plt.title('DNI over Time')
plt.xlabel('Timestamp')
plt.ylabel('DNI (W/m²)')
plt.xticks(rotation=45)
plt.grid(True)

# Plot DHI
plt.subplot(2, 2, 3)
plt.plot(df.index, df['DHI'], label='DHI', color='green')
plt.title('DHI over Time')
plt.xlabel('Timestamp')
plt.ylabel('DHI (W/m²)')
plt.xticks(rotation=45)
plt.grid(True)

# Plot Tamb
plt.subplot(2, 2, 4)
plt.plot(df.index, df['Tamb'], label='Tamb', color='red')
plt.title('Tamb over Time')
plt.xlabel('Timestamp')
plt.ylabel('Tamb (°C)')
plt.xticks(rotation=45)
plt.grid(True)

# Adjust layout to avoid overlap
plt.tight_layout()
plt.savefig('notebooks/togo/timeseries_plots.png')
plt.show()

In [ ]:
# Analyze patterns by month
monthly_avg = df.groupby(df.index.month)[['GHI', 'DNI', 'DHI', 'Tamb']].mean()
print("Monthly Averages:\n", monthly_avg)

In [ ]:
# Analyze patterns by hour of the day
hourly_avg = df.groupby(df.index.hour)[['GHI', 'DNI', 'DHI']].mean()
print("\nHourly Averages:\n", hourly_avg)

# Look for anomalies (e.g., GHI spikes)
ghi_spikes = df[df['GHI'] > df['GHI'].quantile(0.99)]  # Top 1% of GHI values
print("\nPotential GHI Anomalies (Spikes):\n", ghi_spikes[['GHI']])

 Next Step: Cleaning Impact Analysis
 Objective: Analyze the impact of the Cleaning flag on ModA and ModB, 

 Plan:

 Group the data by the Cleaning flag.
 Compute the mean of ModA and ModB for each group (pre- and post-cleaning).
 Plot a bar chart to compare the averages.

In [ ]:


# Group by Cleaning flag and calculate mean for ModA and ModB
cleaning_impact = df.groupby('Cleaning')[['ModA', 'ModB']].mean()
print("Average ModA and ModB by Cleaning Flag:\n", cleaning_impact)

# Plot the impact of cleaning on ModA and ModB
cleaning_impact_melted = cleaning_impact.reset_index().melt(id_vars='Cleaning', value_vars=['ModA', 'ModB'], var_name='Module', value_name='Value')
plt.figure(figsize=(8, 6))
sns.barplot(x='Cleaning', y='Value', hue='Module', data=cleaning_impact_melted)
plt.title('Impact of Cleaning on ModA and ModB')
plt.xlabel('Cleaning Flag (0 = Before, 1 = After)')
plt.ylabel('Average Value (W/m²)')
plt.savefig('notebooks/togo/cleaning_impact_plot.png')
plt.show()

In [ ]:
#Next Step: Correlation Analysis
# Objective: Compute the correlation matrix for GHI, DNI, DHI, Tamb, WS, ModA, and ModB to identify relationships, as specified in Task 2. Visualize the correlations using a heatmap.

# Plan:

# Calculate the Pearson correlation matrix for the specified columns.
# Create a heatmap to visualize the correlations.

In [ ]:
# Select columns for correlation analysis
correlation_cols = ['GHI', 'DNI', 'DHI', 'Tamb', 'WS', 'ModA', 'ModB']

# Compute correlation matrix
correlation_matrix = df[correlation_cols].corr()

# Plot heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation Matrix of Solar Variables')
plt.savefig('notebooks/togo/correlation_heatmap.png')
plt.show()

print("Correlation Matrix:\n", correlation_matrix)

Next Step: Wind Rose Plot
 Objective: Create a wind rose or radial bar plot to visualize the distribution of wind speed (WS) and wind direction (WD),

 Plan:

 Use the windrose library to plot WS against WD.
 Ensure WD is in degrees (0–360) and WS is in m/s.

In [ ]:


# Create wind rose plot
fig = plt.figure(figsize=(8, 8))
ax = WindroseAxes.from_ax(fig=fig)
ax.bar(df['WD'], df['WS'], normed=True, opening=0.8, edgecolor='white')
ax.set_legend(title="Wind Speed (m/s)")
plt.title('Wind Rose Plot')

# Create notebooks directory if it doesn't exist
os.makedirs('notebooks', exist_ok=True)
plt.savefig('notebooks/togo/wind_rose_plot.png')
plt.show()

In [ ]:
# Save the cleaned DataFrame to a CSV file
df.to_csv('../data/cleaned/togo_cleaned.csv', index=True)
print("Cleaned dataset saved to 'data/togo_cleaned.csv'")